In [1]:
%reload_ext autoreload
import wave
import os
import numpy as np
import itertools
import op
from util import WavFileParts


In [2]:
## defs
alpha = 0.005
thresholdFactor = 1
minimalSoundTime = 0.15

correlatedTimeDiffThreshold = 0.025  # ~8.5 m
onlySkipOldestPointForCorrelation = False

In [3]:
def lowpass(signal):
    lowpassSig = [0] * len(signal)
    lowpassSig[0] = abs(signal[0])
    abssigTimesAlpha = alpha * np.array(abs(signal))

    for i in range(1, len(abssigTimesAlpha)):
        lowpassSig[i] = (abssigTimesAlpha[i] + (1 - alpha) * lowpassSig[i - 1])
    return lowpassSig

In [4]:
def getSoundThresholdFromSignal(signal):
    lpsig = lowpass(signal)
    return thresholdFactor * np.max(lpsig)

In [5]:
def getSoundThresholdFromFile(filename, start, end):
    wavFile = wave.open(filename, 'rb')
    framerate = wavFile.getframerate()
    startIndex = int(2 * start * framerate)
    endIndex = int(2 * end * framerate)
    signal = np.fromstring(wavFile.readframes(endIndex)[startIndex: endIndex], np.int16)
    wavFile.close()
    return getSoundThresholdFromSignal(signal)

In [6]:
def determineThresholds(microphones,
                        datasetDir='/Volumes/SAA_DATA/datasets/localizationRecordings/20160919',
                        fileDate=160919,
                        fileNum=218,
                        silenceStart=1,
                        silenceEnd=5
                        ):
    thresholds = {}

    filename = datasetDir + '/{:d}_{:d}'.format(fileDate, fileNum) + '_mono{:d}.wav'

    for microphone in microphones:
        wavFileName = filename.format(microphone)
        soundThreshold = getSoundThresholdFromFile(wavFileName, silenceStart, silenceEnd)
        thresholds[microphone] = soundThreshold

    return thresholds

In [17]:
def funcLmb(v):
    return v[0] - v[1]

lpSig = np.array([3,4,6,2,1,6,2,2,5])
indices = np.array(range(len(lpSig)))
indices = indices[lpSig > 2]
print(indices)
# enumerate maakt lijstje met plek in lijst en item zelf
print(list(enumerate(indices)))
print(itertools.groupby(enumerate(indices), key=funcLmb))
ig = itertools.groupby(enumerate(indices), key=funcLmb)
for v,a in ig:
    print(v)

[0 1 2 5 8]
[(0, 0), (1, 1), (2, 2), (3, 5), (4, 8)]
0
-2
-4


In [18]:
def funcLmb(v):
    return v[0] - v[1]

def getSoundChunkIndices(signal, threshold):
    lpSig = lowpass(signal)
    indices = np.array(range(len(signal)))
    indices = indices[lpSig > threshold]

    soundIndexList = []
    for k, g in itertools.groupby(enumerate(indices),funcLmb):
        soundIndexList.append(map(operator.itemgetter(1), g))

    chunkIndices = []

    for chunk in soundIndexList:
        if len(chunk) > 1:
            chunkIndices.append([chunk[0], chunk[-1]])

    return chunkIndices

In [19]:
def getSoundChunks(filename, start, end, soundThreshold):
    timeChunks = []
    wavFile = wave.open(filename, 'rb')
    wavFile.rewind()
    nFrames = wavFile.getnframes()
    framerate = wavFile.getframerate()
    signal = np.fromstring(
        wavFile.readframes(int(end * framerate))[int(start * 2 * framerate):int(end * 2 * framerate)],
        np.int16)  # 2* framerate, because conversion int16
    wavFile.close()

    soundChunks = getSoundChunkIndices(signal, soundThreshold)
    nr = 1

    for chunk in soundChunks:
        startTime = chunk[0] * 1.0 / framerate + start
        endTime = chunk[1] * 1.0 / framerate + start

        if endTime - startTime > minimalSoundTime:
            timeChunks.append([startTime, endTime])

    return timeChunks

In [20]:
def findAndSaveSoundChunks(fileparts,
                           dirName='/Users/etto/Dropbox/git/wavLocalization/results',
                           datasetDir='/Volumes/SAA_DATA/datasets/localizationRecordings/20160919',
                           fileDate=160919,
                           thresholdFileNum=218,
                           thrSilenceStart=1,
                           thrSilenceEnd=5):
    """
    print soundchunks in json format
    store the printout in results/soundchunks.py
    :return:
    """
    microphones = [1, 2, 3, 4]

    # determine thresholds
    thresholds = determineThresholds(microphones, datasetDir=datasetDir,
                                     fileDate=fileDate, fileNum=thresholdFileNum,
                                     silenceStart=thrSilenceStart, silenceEnd=thrSilenceEnd
                                     )  # dictionary: int -> float

    if not os.path.isdir(dirName):
        os.mkdir(dirName)

    filename = dirName + '/soundChunks.py'
    # filename = 'dummy'  # to prevent accidental overwriting soundChunks
    with open(filename, 'w') as out:
        out.write('soundChunks = [\n')
        for filepart in fileparts:  # type: WavFilePart
            for microphone in microphones:
                filename = datasetDir + '/{:d}_{:d}_mono{:d}.wav'.format(fileDate, filepart.fileNr, microphone)
                chunks = getSoundChunks(filename, filepart.getStartSecs(), filepart.getEndSecs(),
                                                            thresholds[microphone])
                filepart.setSoundChunks(microphone, chunks)
            out.write(filepart.toJSON())
            print('written ' + str(filepart))
            out.write(',\n')
        out.write(']\n')

    open(dirName + '/__init__.py', 'w').close()

In [21]:
#def findSoundChunks20180609

osRecordingsDir='/Volumes/SAA_DATA/datasets/localizationRecordings/'
osRecordingsDir = 'E:/SAA_DATA/localizationRecordings/'

scTargetBase = '../storedData/results'
if not os.path.isdir(scTargetBase):
    os.mkdir(scTargetBase)

dirName = scTargetBase + '/Studio'
datasetDir = osRecordingsDir + '20171011'
fileDate = 170816
thresholdFileNum = 745
thrSilenceStart = 1
thrSilenceEnd = 2.5

wavFileParts = WavFileParts.getWavFileParts20171014_Studio()
findAndSaveSoundChunks(wavFileParts,
                               dirName,
                               datasetDir,
                               fileDate,
                               thresholdFileNum,
                               thrSilenceStart,
                               thrSilenceEnd)


dirName = scTargetBase + '/G428'
datasetDir = osRecordingsDir + '20170221'
fileDate = 170221
thresholdFileNum = 542
thrSilenceStart = 14
thrSilenceEnd = 17
wavFileParts = WavFileParts.getWavFileParts20170221_G4_28()

findAndSaveSoundChunks(wavFileParts,
                           dirName,
                           datasetDir,
                           fileDate,
                           thresholdFileNum,
                           thrSilenceStart,
                           thrSilenceEnd)

dirName = scTargetBase + '/G527'
datasetDir = osRecordingsDir + '20170221'
fileDate = 170221
thresholdFileNum = 551
thrSilenceStart = 10
thrSilenceEnd = 15

wavFileParts = WavFileParts.getWavFileParts20170221_G5_27()
findAndSaveSoundChunks(wavFileParts,
                           dirName,
                           datasetDir,
                           fileDate,
                           thresholdFileNum,
                           thrSilenceStart,
                           thrSilenceEnd)


NameError: name 'operator' is not defined